# Portfolio Optimizer

In [5]:
# Initial imports
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime
import alpaca_trade_api as tradeapi

In [2]:
# Load .env enviroment variables
load_dotenv("C:\\Users\\rhnil\\Desktop\\Columbia_Bootcamp\\alpaca.env")


True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [6]:
# Format current date as ISO format [TO BE UPDATED BASED ON DATE OF SMS]
sd = str(datetime.date.today() - datetime.timedelta(30))
ed = str(datetime.date.today())

start_date = pd.Timestamp(sd, tz="America/New_York").isoformat()
end_date = pd.Timestamp(ed, tz="America/New_York").isoformat()

# Set the tickers [TO BE REPLACED with tickers from SMS]
tickers = ["TSLA", "HD", "MRNA", "FB"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get prices from alpaca
portfolio_df = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=35,
).df


NameError: name 'datetime' is not defined

In [ ]:
for column in portfolio_df.columns.get_level_values(1):
    if column !='close':
        portfolio_df = portfolio_df.drop(column, axis=1, level=1)
        
portfolio_df.columns = portfolio_df.columns.get_level_values(0)
portfolio_df.reset_index(inplace=True)

In [ ]:
portfolio_df['time'] = pd.to_datetime(portfolio_df['time'])
portfolio_df['time'] = portfolio_df['time'].dt.date
portfolio_df = portfolio_df.set_index('time')

portfolio_returns_df = portfolio_df.pct_change().dropna()

In [ ]:
# Monte Carlo simulation to identify "optimized" portfolio weights -- "optimized" defined as highest Sharpe ratio.
# Code block learned from Sigma Coding (https://www.youtube.com/channel/UCBsTB02yO0QGwtlfiv5m25Q/about)

# Set number of simulations
no_of_runs = 100
no_of_stocks = len(tickers)

# Initialize portfolio weights to zero
all_weights = np.zeros((no_of_runs, no_of_stocks))

# Initialize arrays to hold results of simulation
return_array = np.zeros(no_of_runs)
vol_array = np.zeros(no_of_runs)
sharpe_array = np.zeros(no_of_runs)

# Calculate log returns of each stock (logs are better matched to a time dimension, i.e. returns overtime)
log_return = np.log(1 + portfolio_returns_df)

# Simulation
for x in range(no_of_runs):
    
    # Step 1: Calculate weights
    weights = np.array(np.random.random(no_of_stocks))
    weights = weights / np.sum(weights)
    
    # Step 2: Add weights to weights array
    all_weights[x,:] = weights
    
    # Step 3: Calculate expected returns
    return_array[x] = np.sum((log_return.mean()*weights)*252)
    
    # Step 4: Calculate volitility and add to vol_array
    vol_array[x] = np.sqrt(np.dot(weights.T, np.dot(log_return.cov()*252, weights)))
    
    # Step 5: Calculate Sharpe ratio
    sharpe_array[x] = return_array[x]/vol_array[x]
    
# Combine all arrays into master array
simulation_data =[return_array, vol_array, sharpe_array, all_weights]

# Create dataframe from master array
simulation_df = pd.DataFrame(data=simulation_data).T

# Name simulsimulation_df columns
simulation_df.columns = ['Returns', 'Volitility', 'Sharpe Ratio', 'Portfolio Weights']

# Ensure sure data types are floats (no accidental strings) 
simulation_df = simulation_df.infer_objects()

In [ ]:
# Find portfolio weights with highest sharpe ratio 
Max_SharpeRatio_row = simulation_df.iloc[simulation_df['Sharpe Ratio'].idxmax()]

In [ ]:
optimized_weights_df = pd.Series(Max_SharpeRatio_row['Portfolio Weights']).to_frame()
optimized_weights_df = optimized_weights_df.T
optimized_weights_df.columns = list(portfolio_returns_df.columns.values)
optimized_weights_df